### VGG 16 모델 살펴 보기

In [ ]:
from torchvision import models

print(models.list_models(include='vgg*'))

In [ ]:
torch_vgg16_bn_model = models.vgg16_bn(weights=None)
print(torch_vgg16_bn_model)

### VGG 16 모델 생성 - 01

In [ ]:
import torch
import torch.nn as nn

IMG_CHANNELS = 3

class VGG16_01(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.conv_block_01 = nn.Sequential(
            nn.Conv2d(in_channels=IMG_CHANNELS, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=64), nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=64), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.conv_block_02 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=128), nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=128), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.conv_block_03 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=256), nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=256), nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=256), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.conv_block_04 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512), nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512), nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.conv_block_05 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512), nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512), nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=512), nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.adaptive_pool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False), 
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=num_classes, bias=True)
        )

    def forward(self, x):
        x = self.conv_block_01(x)
        x = self.conv_block_02(x)
        x = self.conv_block_03(x)
        x = self.conv_block_04(x)
        x = self.conv_block_05(x)

        x = self.adaptive_pool(x)
        x = torch.flatten(x, start_dim=1) # x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

In [ ]:
from torchinfo import summary

my_vgg16_01 = VGG16_01(num_classes=1000)

summary(model=my_vgg16_01, input_size=(1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params'], 
        row_settings=['var_names'])

In [ ]:
from torchinfo import summary

torch_vgg16 = models.vgg16_bn(weights='DEFAULT')

summary(model=torch_vgg16, input_size=(1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params'], 
        row_settings=['var_names'])


### 연속되는 Conv 적용 Sequential을 생성하는 함수를 생성하여 VGG16 모델 구현
* create_convbn_block(in_channels, last_channels, num_convs)는 num_convs 만큼 반복하여 in_channels과 last_channels를 Conv 적용한 Sequential로 반환

In [ ]:
import torch
import torch.nn as nn

IMG_CHANNELS = 3

class VGG16_02(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # Feature Extractor
        self.conv_block_01 = self.create_convbn_block(in_channels=3, last_channels=64, num_convs=2)
        self.conv_block_02 = self.create_convbn_block(64, 128, num_convs=2)
        self.conv_block_03 = self.create_convbn_block(128, 256, num_convs=3)
        self.conv_block_04 = self.create_convbn_block(256, 512, num_convs=3)
        self.conv_block_05 = self.create_convbn_block(512, 512, num_convs=3)

        #GAP와 Classifier Layer
        self.adaptive_pool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=num_classes, bias=True)
        )

    def create_convbn_block(self, in_channels, last_channels, num_convs=2):
        if num_convs == 2:
            conv_bn_block = nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=last_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(last_channels), nn.ReLU(),
                nn.Conv2d(in_channels=last_channels, out_channels=last_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(last_channels), nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
        elif num_convs == 3:
            conv_bn_block = nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=last_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(last_channels), nn.ReLU(),
                nn.Conv2d(in_channels=last_channels, out_channels=last_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(last_channels), nn.ReLU(),
                nn.Conv2d(in_channels=last_channels, out_channels=last_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(last_channels), nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
        
        return conv_bn_block

    def forward(self, x):
        # Feature Extractor forward
        x = self.conv_block_01(x)
        x = self.conv_block_02(x)
        x = self.conv_block_03(x)
        x = self.conv_block_04(x)
        x = self.conv_block_05(x)

        # GAP, 마지막 Classifier forward
        x = self.adaptive_pool(x)
        x = torch.flatten(x, start_dim=1) #x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

In [ ]:
my_vgg16_02 = VGG16_02(num_classes=1000)

summary(model=my_vgg16_02, input_size=(1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params'], 
        row_settings=['var_names'])

### 연속되는 Conv block을 가지는 VGGBlock 서브 모듈을 만들고 이를 활용하여 VGG16 모델 구현
* ConvBlock은 Conv->BN->ReLU 구성
* VGGBlock은 연속되는 ConvBlock을 list에 담고 Sequential로 연결함

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
    def forward(self, x):
        return self.block(x)

class VGGBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_convs):
        super().__init__()
        # num_convs 값에 따라 ConvBlock을 담아둘 list
        layers = []
        for i in range(num_convs):
            layers.append(ConvBlock(in_channels, out_channels))
            in_channels = out_channels
        layers.append(nn.MaxPool2d(kernel_size=2))
        # layers list에 있는 모든 Layer들을 Sequential로 담아서 연결
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            VGGBlock(3, 64, num_convs=2),
            VGGBlock(64, 128, num_convs=2),
            VGGBlock(128, 256, num_convs=3),
            VGGBlock(256, 512, num_convs=3),
            VGGBlock(512, 512, num_convs=3),
        )
        #GAP와 Classifier Layer
        self.adaptive_pool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=num_classes, bias=True)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

    

In [ ]:
vgg16_model = VGG16(num_classes=1000)
print(vgg16_model)

In [ ]:
from torchinfo import summary 

summary(model=vgg16_model, input_size=(1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params'], 
        depth=4, #depth=3 이 기본. 이 경우 ConvBlock의 내부 Layer가 보이지 않음
        row_settings=['var_names'])

### VGG16 모델 학습 및 평가
* Flowers 데이터 세트로 학습 및 평가
* Trainer 클래스는 Modular 활용

In [ ]:
!ls /kaggle/input

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

def create_flowers_meta_df(file_dir):
    paths = [] # 이미지 파일 경로 리스트
    labels = [] # 꽃 종류
    
    # os.walk()를 이용하여 특정 디렉토리 밑에 있는 모든 하위 디렉토리를 모두 조사. 
    # kaggle/input/flowers-dataset 하위 디렉토리 밑에 jpg 확장자를 가진 파일이 모두 이미지 파일임
    # kaggle/input/flowers-dataset 밑으로 하위 디렉토리 존재
    for dirname, _, filenames in os.walk(file_dir):
        for filename in filenames:
            # 이미지 파일이 아닌 파일도 해당 디렉토리에 있음.
            if '.jpg' in filename:
                # 파일의 절대 경로를 file_path 변수에 할당. 
                file_path = dirname+'/'+ filename
                paths.append(file_path)
                
                # 파일의 절대 경로에 daily, dandelion, roses, sunflowers, tulips에 따라 labels에 값 할당.               label_gubuns.append('daisy')
                if 'daisy' in file_path:
                    labels.append('daisy')
                elif 'dandelion' in file_path:
                    labels.append('dandelion')
                elif 'roses' in file_path:
                    labels.append('rose')
                elif 'sunflowers' in file_path:
                    labels.append('sunflowers')
                elif 'tulips' in file_path:
                    labels.append('tulips')
    # DataFrame 메타 데이터 생성. 
    data_df = pd.DataFrame({'path':paths, 
                            'label':labels})
    # Target값  변환
    label_mapping = {'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflowers': 3, 'tulips': 4}
    data_df['target'] = data_df['label'].map(label_mapping)

    return data_df

In [ ]:
from sklearn.model_selection import train_test_split

data_df = create_flowers_meta_df('/kaggle/input/flowers-dataset') # /kaggle/input

# 전체 데이터 세트에서 학습(전체의 70%)과 테스트용(전체의 30%) 메타 정보 DataFrame 생성.
train_df, test_df = train_test_split(data_df, test_size=0.3, stratify=data_df['target'], random_state=2025)
# 기존 학습 DataFrame을 다시 학습과 검증 DataFrame으로 분할. 80%가 학습, 20%가 검증
tr_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['target'], random_state=2025)

print(data_df.shape, train_df.shape, tr_df.shape, val_df.shape, test_df.shape)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image

BATCH_SIZE = 16

class FlowerDataset(Dataset):
    # 이미지 파일리스트, 타겟 파일리스트, transforms 등 이미지와 타겟 데이터 가공에 필요한 인자들을 입력 받음
    def __init__(self, image_paths, targets=None, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform
    
    # 전체 건수를 반환
    def __len__(self):
        return len(self.image_paths)
        
    # idx로 지정된 하나의 image, label을 tensor 형태로 반환
    def __getitem__(self, idx):    
        # PIL을 이용하여 이미지 로딩하고 PIL Image 객체 반환.
        pil_image = Image.open(self.image_paths[idx])
        # 보통은 transform이 None이 되는 경우는 거의 없음(Tensor 변환이라도 있음)
        image = self.transform(pil_image)

        if self.targets is not None:
            # 개별 target값을 tensor로 변환.
            target = torch.tensor(self.targets[idx])
            return image, target
        # 테스트 데이터의 경우 targets가 입력 되지 않을 수 있으므로 이를 대비. 
        else:
            return image

def create_tr_val_loader(tr_df, val_df, tr_transform, val_transform):
    tr_dataset = FlowerDataset(image_paths=tr_df['path'].to_list(), 
                               targets=tr_df['target'].to_list(), transform=tr_transform)
    val_dataset = FlowerDataset(image_paths=val_df['path'].to_list(), 
                                targets=val_df['target'].to_list(), transform=val_transform)
    
    tr_loader = DataLoader(tr_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=2*BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    return tr_loader, val_loader

In [ ]:
IMG_SIZE = 224
IMG_MEANS = [0.485, 0.456, 0.406] # ImageNet 데이터세트의 이미지 채널별 평균값
IMG_STD = [0.229, 0.224, 0.225] # ImageNet 데이터세트의 이미지 채널별 표준편차값

tr_transform = T.Compose([
            T.RandomHorizontalFlip(p=0.3),
            T.RandomVerticalFlip(p=0.3),
            T.RandomApply([T.CenterCrop(size=(200, 200))], p=0.4),
            T.Resize(size=(IMG_SIZE, IMG_SIZE)),
            T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
])

val_transform = T.Compose([
            T.Resize(size=(IMG_SIZE, IMG_SIZE)),
            T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
])

tr_loader, val_loader = create_tr_val_loader(tr_df=tr_df, val_df=val_df, 
                                             tr_transform=tr_transform, val_transform=val_transform)
images, labels = next(iter(tr_loader))
print(images.shape, labels.shape)

#### Trainer 클래스 생성 및 적용
* Trainer 클래스는 https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/main/modular/v1/utils.py?raw=true 로 download 후 import 함

In [ ]:
# /kaggle/working/modular/v1 디렉토리에 utils.py 파일 다운로드
!rm -rf ./modular/v1
!mkdir -p ./modular/v1
!wget -O ./modular/v1/utils.py https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/main/modular/v1/utils.py?raw=true
!ls ./modular/v1

import sys

# 반드시 system path를 아래와 같이 잡아줘야 함. 
sys.path.append('/kaggle/working')

#아래가 수행되는지 반드시 확인
from modular.v1.utils import Trainer, Predictor, ModelCheckpoint, EarlyStopping

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 5개의 꽃 종류
NUM_CLASSES = 5

# 직접 구현한 VGG16 모델 생성
model = VGG16(num_classes=NUM_CLASSES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = Adam(model.parameters(), lr=0.0001) #lr = 1e-4
loss_fn = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(
            optimizer=optimizer, mode='min', factor=0.5, patience=5, threshold=0.01, min_lr=1e-6)

trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
       train_loader=tr_loader, val_loader=val_loader, scheduler=scheduler, device=device)
# 학습 및 평가.
history = trainer.fit(60)

In [ ]:
from modular.v1.utils import Predictor

test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

IMG_SIZE=224
test_transform = T.Compose([
                        T.Resize(size=(IMG_SIZE, IMG_SIZE)),
                        T.ToTensor(), 
                        T.Normalize(mean=[0.485, 0.456, 0.406], 
                                    std=[0.229, 0.224, 0.225])
])

test_dataset = FlowerDataset(image_paths=test_image_paths, 
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')